For a smooth run, Save the file I shared on google in the following location on your drive:

/content/drive/MyDrive/DataFiles/

You will have to make a DataFiles folder in your drive home page and then add the shared folder

# Setup

In [ ]:
GITHUB_TOKEN = 'fe2e680f071553cddb5f698cc58373a5106380d4'
command = f'git clone https://{GITHUB_TOKEN}@github.com/shmulvad/nlp-project.git'
!{command}

%cd nlp-project/src

Cloning into 'nlp-project'...
remote: Enumerating objects: 391, done.
remote: Counting objects: 100% (391/391), done.
remote: Compressing objects: 100% (276/276), done.
remote: Total 391 (delta 168), reused 320 (delta 100), pack-reused 0
Receiving objects: 100% (391/391), 377.19 KiB | 2.81 MiB/s, done.
Resolving deltas: 100% (168/168), done.
/content/nlp-project/src


In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh > /dev/null | sudo bash
!sudo apt-get install git-lfs -qq > /dev/null
!git lfs install

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Updated git hooks.
Git LFS initialized.


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 542kB 2.8MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-cp37-none-any.whl size=541554 sha256=3af979eeaef9e17990149c55981be08776dea2c1980281a15f4eb9d4061ce2fa
  Stored in directory: /root/.cache/pip/wheels/22/46/06/9b6d10ed02c85e93c3bb33ac50e2d368b2586248f192a2e22a
Successfully built wordninja


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/DataFiles/nlp-project-main/src/featurizer_colab.py .
!cp //content/drive/MyDrive/DataFiles/nlp-project-main/src/read_data_colab.py .
!cp /content/drive/MyDrive/DataFiles/nlp-project-main/src/url_tokenizer.py .
!cp /content/drive/MyDrive/DataFiles/nlp-project-main/src/util.py .

!pip install wordninja

In [ ]:
import os
import csv
import re
import string
import pickle

from tqdm import tqdm
import numpy as np
import pandas as pd

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from url_tokenizer import url_tokenizer
from featurizer_colab import UrlFeaturizer, GLOVE, CONCEPTNET, SAMPLE

### Reading DataFrames

In [ ]:
with open('nlp-data/datasets.pkl', 'rb') as f:
  datasets = pickle.load(f)

dmoz = datasets['dmoz']
phishing = datasets['phishing']
ilp = datasets['ilp']

### Generate Feature Vector

This part is only to be used if you want to re-engineer the feature Vector. Since this is an expensive task, I have saved the feature vectors in the initially decided engineering and made them available to be loaded directly

In [ ]:
UF = UrlFeaturizer(GLOVE)

Reading the GloVe word vector file...


KeyboardInterrupt: ignored

In [ ]:
url_regex = re.compile(r'''
        (https?):\/\/                                   # http s
        ([-a-zA-Z0-9@:%._\+~#=]+\.[a-zA-Z0-9()]{1,12})  # domains
        \b
        ([-a-zA-Z0-9()@:%_\+;.~#&//=]*)                 # path
        \??
        ([-a-zA-Z0-9()@:%_\+;.~#&//=?]*)                # args
    ''', re.DOTALL | re.VERBOSE)

def GenerateFeatureVector(df, UF):
    df.reset_index(drop = True, inplace = True)
    df = pd.get_dummies(data=df, columns=['label'])
    label = df.drop(['idx','url'],axis=1).values

    url_features = list()
    labels = list()

    inputs1 = Input(shape=(31, 300))
    lstm1 = LSTM(100)(inputs1)
    model = Model(inputs=inputs1, outputs=lstm1)

    for i in tqdm(range(df.shape[0])):
        url = df.iloc[i].url
        match = url_regex.match(url.lower())
        if match:
            features = UF.featurize(url)
            temp =   model.predict(features[1].reshape(1,31,300)).reshape(100)
            labels.append(label[i])
            url_features.append(np.concatenate((temp,features[0])))

    url_features = np.array(url_features)
    labels = np.array(labels)
    return url_features,labels

Change the df value to the Dataframe you want to generate features and labels for 

In [ ]:
df = phishing

In [ ]:
url_features,labels = GenerateFeatureVector(df,UF)

np.save('url_features_phishing.npy', url_features)
np.save('labels_phishing.npy', labels)

### Load feature Vector

Here you can directly load the generated feature vectors

#### Load dmoz vectors

In [ ]:
url_features_dmoz = np.load('/content/drive/MyDrive/DataFiles/nlp-project-main/src/Pickled_Vectors/url_features_dmoz.npy')
labels_dmoz = np.load('/content/drive/MyDrive/DataFiles/nlp-project-main/src/Pickled_Vectors/labels_dmoz.npy')

#### Load phishing vectors

In [ ]:
url_features_phishing = np.load('/content/drive/MyDrive/DataFiles/nlp-project-main/src/Pickled_Vectors/url_features_phishing.npy')
labels_phishing = np.load('/content/drive/MyDrive/DataFiles/nlp-project-main/src/Pickled_Vectors/labels_phishing.npy')

# Phishing Model


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(url_features_phishing,labels_phishing,test_size=0.2)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,Y_train)
Y_RF_pred=rf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Y_test,Y_RF_pred))

              precision    recall  f1-score   support

           0       0.45      0.46      0.45     11064
           1       0.12      0.12      0.12      7073

   micro avg       0.33      0.33      0.33     18137
   macro avg       0.29      0.29      0.29     18137
weighted avg       0.32      0.33      0.32     18137
 samples avg       0.33      0.33      0.33     18137



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train,Y_train)
Y_SVM_pred=svc.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Y_test,Y_SVM_pred))

              precision    recall  f1-score   support

           0       0.45      0.00      0.01      7052
           1       0.61      1.00      0.76     11085

    accuracy                           0.61     18137
   macro avg       0.53      0.50      0.38     18137
weighted avg       0.55      0.61      0.47     18137



## Deep Network

In [ ]:
phishing_features_deep = url_features_phishing.reshape(*url_features_phishing.shape, 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(phishing_features_deep,labels_phishing,test_size=0.2)

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Bidirectional, TimeDistributed, Conv1D, ZeroPadding1D, GRU
from tensorflow.keras.layers import Lambda, Input, Dropout, Masking, BatchNormalization, Activation
from tensorflow.keras.models import Model

def cnn_lstm(input_dim, output_dim, dropout=0.2, n_layers=1):
    dtype = 'float64'
    input_data = Input(name='the_input', shape=input_dim, dtype=dtype)
    x = Conv1D(filters=256, kernel_size=10, strides=4, name='conv_1')(input_data)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout, name='dropout_1')(x)
        
    x = LSTM(128, activation='relu', return_sequences=True,dropout=dropout, name='lstm_1')(x)
    x = LSTM(128, activation='relu', return_sequences=False,dropout=dropout, name='lstm_2')(x)

    # 1 fully connected layer DNN ReLu with default 20% dropout
    x = Dense(units=64, activation='relu', name='fc')(x)
    x = Dropout(dropout, name='dropout_2')(x)

    # Output layer with softmax
    y_pred = Dense(units=output_dim, activation='softmax', name='softmax')(x)
    model = Model(inputs=input_data, outputs=y_pred)   
    return model

input_dim = (107,1)
classes = 2
K.clear_session()
model = cnn_lstm(input_dim, classes)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, 107, 1)]          0         
_________________________________________________________________
conv_1 (Conv1D)              (None, 25, 256)           2816      
_________________________________________________________________
batch_normalization (BatchNo (None, 25, 256)           1024      
_________________________________________________________________
activation (Activation)      (None, 25, 256)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 128)           197120    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard

sgd = SGD(lr=0.00001, clipnorm=1.0)
adam = Adam(lr=1e-4, clipnorm=1.0)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=128, epochs=10,
                    validation_data=(X_test, Y_test),
                    callbacks=[TensorBoard(log_dir='logs',
                                           histogram_freq=1,
                                           update_freq='epoch')])

Epoch 1/10
567/567 [==============================] - 119s 153ms/step - loss: 0.6431 - accuracy: 0.5997 - val_loss: 0.5912 - val_accuracy: 0.6079
Epoch 2/10
567/567 [==============================] - 84s 149ms/step - loss: 0.5857 - accuracy: 0.6045 - val_loss: 0.5715 - val_accuracy: 0.6074
Epoch 3/10
567/567 [==============================] - 84s 149ms/step - loss: 0.5796 - accuracy: 0.6061 - val_loss: 0.5679 - val_accuracy: 0.6118
Epoch 4/10
567/567 [==============================] - 84s 147ms/step - loss: 0.5754 - accuracy: 0.6069 - val_loss: 0.5663 - val_accuracy: 0.6110
Epoch 5/10
567/567 [==============================] - 84s 149ms/step - loss: 0.5734 - accuracy: 0.6048 - val_loss: 0.5664 - val_accuracy: 0.6123
Epoch 6/10
567/567 [==============================] - 84s 149ms/step - loss: 0.5721 - accuracy: 0.6060 - val_loss: 0.5645 - val_accuracy: 0.6079
Epoch 7/10
567/567 [==============================] - 84s 148ms/step - loss: 0.5718 - accuracy: 0.6056 - val_loss: 0.5632 - val_a

# URL Classification Model

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(url_features_dmoz,labels_dmoz,test_size=0.2)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,Y_train)
Y_RF_pred=rf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Y_test,Y_RF_pred))

### SVM

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train,Y_train)
Y_SVM_pred=svc.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Y_test,Y_SVM_pred))

              precision    recall  f1-score   support

           0       0.45      0.00      0.01      7052
           1       0.61      1.00      0.76     11085

    accuracy                           0.61     18137
   macro avg       0.53      0.50      0.38     18137
weighted avg       0.55      0.61      0.47     18137



## Deep Network

In [ ]:
dmoz_features_deep = url_features_dmoz.reshape(url_features_dmoz.shape[0],url_features_dmoz.shape[1],1)


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(dmoz_features_deep,labels_dmoz,test_size=0.2)

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Bidirectional, TimeDistributed, Conv1D, ZeroPadding1D, GRU
from tensorflow.keras.layers import Lambda, Input, Dropout, Masking, BatchNormalization, Activation
from tensorflow.keras.models import Model

def cnn_lstm(input_dim, output_dim, dropout=0.2, n_layers=1):
    dtype = 'float64'
    input_data = Input(name='the_input', shape=input_dim, dtype=dtype)
#     x = Conv1D(filters=256, kernel_size=10, strides=4, name='conv_1')(input_data)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = Dropout(dropout, name='dropout_1')(x)
        
    x = LSTM(128, activation='relu', return_sequences=True,dropout=dropout, name='lstm_1')(input_data)
    x = LSTM(128, activation='relu', return_sequences=False,dropout=dropout, name='lstm_2')(x)

    # 1 fully connected layer DNN ReLu with default 20% dropout
    x = Dense(units=64, activation='relu', name='fc')(x)
    x = Dropout(dropout, name='dropout_2')(x)

    # Output layer with softmax
    y_pred = Dense(units=output_dim, activation='softmax', name='softmax')(x)
    model = Model(inputs=input_data, outputs=y_pred)   
    return model

input_dim = (67,1)
classes = 15
K.clear_session()
model = cnn_lstm(input_dim, classes)
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard

sgd = SGD(lr=0.00001, clipnorm=1.0)
adam = Adam(lr=1e-4, clipnorm=1.0)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    batch_size=128, epochs=10,
#                     validation_data=(X_val, Y_val),
                    callbacks=[TensorBoard(log_dir='logs',
                                           histogram_freq=1,
                                           update_freq='epoch')])